<a href="https://colab.research.google.com/github/masoomehmashayekhi/AI-Exercises/blob/main/NLPTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q pandas scikit-learn matplotlib
!pip install -q sentence-transformers
!pip install -q umap-learn
!pip install -q huggingface_hub

In [3]:
!pip install "hazm[full]" --no-deps && pip install numpy==1.26.4 fasttext-wheel flashtext gensim==4.3.3 nltk python-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.4 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=7885574c4a084d125cae4feed302032b5e2e2311c8a67b1560d2659d4ff532a1
  Stored in directory: /root/.cache/pip/wheels/8c/24/da/4d994d7a27cfc73a4e513a669fbeec4a7

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'train.csv', 'validation': 'validation.csv', 'test': 'test.csv'}
df_train = pd.read_csv("hf://datasets/ParsiAI/snappfood-sentiment-analysis/" + splits["train"])
df_validation = pd.read_csv("hf://datasets/ParsiAI/snappfood-sentiment-analysis/" + splits["validation"])
df_test = pd.read_csv("hf://datasets/ParsiAI/snappfood-sentiment-analysis/" + splits["test"])

In [5]:
print("TrainSet Shape:" ,df_train.shape)
print("ValidationSet Shape:" ,df_validation.shape)
print("TestSet Shape:" ,df_test.shape)

TrainSet Shape: (52110, 3)
ValidationSet Shape: (8337, 3)
TestSet Shape: (9033, 3)


In [6]:
df_train.head()

,comment,label,label_id
0,غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است,SAD,1.0
1,بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم,HAPPY,0.0
2,غذا بد بود حالم خیییییلی بده. دل دردو دل پیچه....,SAD,1.0
3,با سلام سابق بر این بسته بندی از کیفیت بهتری ب...,SAD,1.0
4,سلام، خیلی ممنون و متشکرم,HAPPY,0.0


In [7]:
df_train['label_id'].value_counts().sort_index()

,count
label_id,
0.0,26236
1.0,25874


In [8]:
print(df_train['comment'].isna().any())
print(df_train['label_id'].isna().any())

False
False


In [2]:
from hazm import *

normalizer = Normalizer()
df_train['comment_clean'] = df_train['comment'].astype(str).apply(normalizer.normalize)
df_validation['comment_clean'] = df_validation['comment'].astype(str).apply(normalizer.normalize)
df_test['comment_clean'] = df_test['comment'].astype(str).apply(normalizer.normalize)
df_train.head()

,comment,label,label_id,comment_clean
0,غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است,SAD,1.0,غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است
1,بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم,HAPPY,0.0,بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم
2,غذا بد بود حالم خیییییلی بده. دل دردو دل پیچه....,SAD,1.0,غذا بد بود حالم خیلی بده. دل دردو دل‌پیچه. معل...
3,با سلام سابق بر این بسته بندی از کیفیت بهتری ب...,SAD,1.0,با سلام سابق بر این بسته بندی از کیفیت بهتری ب...
4,سلام، خیلی ممنون و متشکرم,HAPPY,0.0,سلام، خیلی ممنون و متشکرم


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

tfidf_vectorixer= TfidfVectorizer(ngram_range=(1,2),
                                  min_df=5,
                                  max_df=0.9)

y_train=df_train['label_id']
y_test =df_test['label_id']
y_validation =df_validation['label_id']

X_train = tfidf_vectorixer.fit_transform(df_train['comment_clean'])
X_validation = tfidf_vectorixer.transform(df_validation['comment_clean'])
X_test = tfidf_vectorixer.transform(df_test['comment_clean'])
X_train.shape


(52110, 29611)

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def accuracy(str,cl):
  y_pred = cl.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)

  # Precision
  precision = precision_score(y_test, y_pred, average='weighted')

  # Recall
  recall = recall_score(y_test, y_pred, average='weighted')

  # F1-score
  f1 = f1_score(y_test, y_pred, average='weighted')

  # Confusion matrix
  cm = confusion_matrix(y_test, y_pred)

  # گزارش کامل
  report = classification_report(y_test, y_pred)

  print(f"--------{str} scores-----")
  print(f"-------------------------------------")
  print("Accuracy :", accuracy)
  print("Precision:", precision)
  print("Recall   :", recall)
  print("F1-score :", f1)
  print("\nConfusion Matrix:\n", cm)
  print("\nClassification Report:\n", report)

In [23]:
from sklearn.linear_model import LogisticRegression


cl= LogisticRegression(max_iter=3000)
cl.fit(X_train,y_train)
accuracy("LogisticRegression",cl)

--------LogisticRegression scores-----
-------------------------------------
Accuracy : 0.8608435735636001
Precision: 0.8631162215360844
Recall   : 0.8608435735636001
F1-score : 0.8605805045767416

Confusion Matrix:
 [[3676  810]
 [ 447 4100]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.82      0.85      4486
         1.0       0.84      0.90      0.87      4547

    accuracy                           0.86      9033
   macro avg       0.86      0.86      0.86      9033
weighted avg       0.86      0.86      0.86      9033



In [26]:
from sklearn.naive_bayes import MultinomialNB


svm= MultinomialNB()
svm.fit(X_train,y_train)
accuracy("MultinomialNB",svm)

--------MultinomialNB scores-----
-------------------------------------
Accuracy : 0.8522085685818664
Precision: 0.8575323765904226
Recall   : 0.8522085685818664
F1-score : 0.8515821473203463

Confusion Matrix:
 [[3540  946]
 [ 389 4158]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.79      0.84      4486
         1.0       0.81      0.91      0.86      4547

    accuracy                           0.85      9033
   macro avg       0.86      0.85      0.85      9033
weighted avg       0.86      0.85      0.85      9033



In [27]:
from sklearn.ensemble import RandomForestClassifier


rf= RandomForestClassifier(n_estimators=300)
rf.fit(X_train,y_train)
accuracy("RandomForestClassifier",rf)

--------RandomForestClassifier scores-----
-------------------------------------
Accuracy : 0.8529835049263811
Precision: 0.8567818578171073
Recall   : 0.8529835049263811
F1-score : 0.8525286363137132

Confusion Matrix:
 [[3586  900]
 [ 428 4119]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.80      0.84      4486
         1.0       0.82      0.91      0.86      4547

    accuracy                           0.85      9033
   macro avg       0.86      0.85      0.85      9033
weighted avg       0.86      0.85      0.85      9033



In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
print(tf.config.list_physical_devices('GPU'))

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['comment_clean'])

sequences = tokenizer.texts_to_sequences(df_train['comment_clean'])
X_train = pad_sequences(sequences, maxlen=50, padding='post', truncating='post')

y_train = to_categorical(df_train['label_id'])


sequences_test = tokenizer.texts_to_sequences(df_test['comment_clean'])
X_test = pad_sequences(sequences_test, maxlen=50, padding='post', truncating='post')

y_test = to_categorical(df_test['label_id'])

model = Sequential([
    Embedding(5000, 128, input_length=50),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


[]


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:

history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

loss, acc = model.evaluate(X_test, y_test)

Epoch 1/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 291s 96ms/step - accuracy: 0.8044 - loss: 0.4258 - val_accuracy: 0.8570 - val_loss: 0.3298
Epoch 2/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 279s 95ms/step - accuracy: 0.8741 - loss: 0.3019 - val_accuracy: 0.8593 - val_loss: 0.3250
Epoch 3/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 326s 96ms/step - accuracy: 0.8895 - loss: 0.2691 - val_accuracy: 0.8542 - val_loss: 0.3358
Epoch 4/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 282s 96ms/step - accuracy: 0.9034 - loss: 0.2407 - val_accuracy: 0.8507 - val_loss: 0.3761
Epoch 5/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 278s 95ms/step - accuracy: 0.9109 - loss: 0.2192 - val_accuracy: 0.8490 - val_loss: 0.3681
Epoch 6/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 329s 97ms/step - accuracy: 0.9285 - loss: 0.1862 - val_accuracy: 0.8490 - val_loss: 0.4356
Epoch 7/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 309s 93ms/step - accuracy: 0.9374 - loss: 0.1590 - val_accuracy: 0.8350 - val_loss: 0.4859
Epoch 8/10
2932/2932 ━━━━━━━━━━━━━━━━━━━━ 284s 97ms/step - accuracy: 

In [37]:
y_pred_rnn = model.predict(X_test)
y_test_prob = np.argmax(y_test, axis=1)


y_pred_rnn_prob = np.argmax(y_pred_rnn, axis=1)

acc_rnn = accuracy_score(y_test_prob, y_pred_rnn_prob)
p_rnn, r_rnn, f1_rnn, _ = precision_recall_fscore_support(y_test_prob, y_pred_rnn_prob, average='weighted')


cm_rnn = confusion_matrix(y_test_prob, y_pred_rnn_prob)

283/283 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step


In [38]:
print(f"--------RNN scores-----")
print(f"---------------------")
print("Accuracy :", acc_rnn)
print("Precision:", p_rnn)
print("Recall   :", r_rnn)
print("F1-score :", f1_rnn)
print("\nConfusion Matrix:\n", cm_rnn)

--------RNN scores-----
---------------------
Accuracy : 0.8342743274659582
Precision: 0.834574433619261
Recall   : 0.8342743274659582
F1-score : 0.8342150467371147

Confusion Matrix:
 [[3666  820]
 [ 677 3870]]


In [4]:
from datasets import Dataset

label2id = {"negative": 0, "positive": 1}
id2label = {0: "negative", 1: "positive"}

dataset = Dataset.from_pandas(df_train)
datatest = Dataset.from_pandas(df_test)

dataset

Dataset({
    features: ['comment', 'label', 'label_id', 'comment_clean'],
    num_rows: 52110
})

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


model_name = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def preprocess_function(examples):
    return tokenizer(examples["comment_clean"], truncation=True, padding=True, max_length=128)

tokenized_dataset_train = dataset.map(preprocess_function, batched=True)
tokenized_dataset_test = datatest.map(preprocess_function, batched=True)


Map:   0%|          | 0/52110 [00:00<?, ? examples/s]

Map:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [7]:
tokenized_train = tokenized_dataset_train.map(lambda x: {"labels": int(x["label_id"])})
tokenized_test  = tokenized_dataset_test.map(lambda x: {"labels": int(x["label_id"])})

Map:   0%|          | 0/52110 [00:00<?, ? examples/s]

Map:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [9]:
cols_to_remove = ['comment', 'comment_clean', 'label', 'label_id']

tokenized_train = tokenized_train.remove_columns([c for c in cols_to_remove if c in tokenized_train.column_names])
tokenized_test  = tokenized_test.remove_columns([c for c in cols_to_remove if c in tokenized_test.column_names])

In [10]:

tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 52110
})

In [11]:
tokenized_train.column_names
tokenized_train[0]

{'input_ids': tensor([    2,  4150,  2712,  3657,  1976,  1921, 15944,  3402,  2121,  2712,
          2067,  1933,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1, "precision": p, "recall": r}

In [24]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./sentiment_farsi_model",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True,
    dataloader_num_workers=4,
    report_to="none",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-846834261.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
Device: Tesla T4


In [25]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.495800,0.588275,0.573121,0.483558,0.726860,0.573121
2,0.550300,0.584099,0.720469,0.714617,0.741746,0.720469
3,0.647100,0.659834,0.559393,0.461593,0.730904,0.559393
4,0.669100,0.649777,0.613085,0.577374,0.674682,0.613085
5,0.668300,0.756549,0.492970,0.412640,0.479162,0.492970


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: T

TrainOutput(global_step=16285, training_loss=0.6208126313048782, metrics={'train_runtime': 2065.2893, 'train_samples_per_second': 126.157, 'train_steps_per_second': 7.885, 'total_flos': 1.7138396368512e+16, 'train_loss': 0.6208126313048782, 'epoch': 5.0})

In [27]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


{'eval_loss': 0.5840990543365479,
 'eval_accuracy': 0.7204693900143917,
 'eval_f1': 0.7146172259869477,
 'eval_precision': 0.7417457458341383,
 'eval_recall': 0.7204693900143917,
 'eval_runtime': 13.7812,
 'eval_samples_per_second': 655.456,
 'eval_steps_per_second': 20.535,
 'epoch': 5.0}